In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

c:\Users\HP\Documents\repos\mlops-regression-mlflow\.venv\Scripts\python.exe
3.0.4
c:\Users\HP\Documents\repos\mlops-regression-mlflow\.venv\Lib\site-packages\xgboost\__init__.py


In [1]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

c:\Users\HP\Documents\repos\mlops-regression-mlflow\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("C://Users//HP//Documents//repos//mlops-regression-mlflow//data//processed//feature_engineered_train.csv")
eval_df  = pd.read_csv("C://Users//HP//Documents//repos//mlops-regression-mlflow//data//processed//feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [3]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [6]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri(
    "file:C:/Users/HP/Documents/repos/mlops-regression-mlflow/mlruns"
)
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/12/19 14:27:36 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-12-19 14:27:36,259] A new study created in memory with name: no-name-791446e9-28a8-412f-ba67-b4153c157e45
[I 2025-12-19 14:27:48,239] Trial 0 finished with value: 73413.60135859973 and parameters: {'n_estimators': 310, 'max_depth': 7, 'learning_rate': 0.02731245016844747, 'subsample': 0.6107195238536401, 'colsample_bytree': 0.8271870960455705, 'min_child_weight': 5, 'gamma': 3.3574285825670263, 'reg_alpha': 0.03200798017545273, 'reg_lambda': 1.9848359883660755e-06}. Best is trial 0 with value: 73413.60135859973.
[I 2025-12-19 14:27:57,157] Trial 1 finished with value: 74518.3001979423 and parameters: {'n_estimators': 646, 'max_depth': 3, 'learning_rate': 0.18519941135160328, 'subsample': 0.9358002759305685, 'colsample_bytree': 0.8366930524264421, 'min_child_weight': 2, 'gamma': 4.706469784615742, 'reg_alpha': 0.9663581619905518, 'reg_lambda':

Best params: {'n_estimators': 477, 'max_depth': 8, 'learning_rate': 0.014184319170680961, 'subsample': 0.6918577826825331, 'colsample_bytree': 0.6153484506153258, 'min_child_weight': 4, 'gamma': 3.0843357334937367, 'reg_alpha': 7.590665773108126e-05, 'reg_lambda': 0.05078616484163577}


In [7]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 32466.737840847156
RMSE: 70982.95641201285
R²: 0.9610624879555727


c:\Users\HP\Documents\repos\mlops-regression-mlflow\.venv\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [14:33:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/12/19 14:33:29 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/12/19 14:33:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
